In [40]:
%load_ext autoreload
%autoreload 2

In [41]:
import os
import pandas as pd

In [42]:
import lib
import plotting

In [43]:
from multiprocessing import Pool

In [10]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score, calinski_harabaz_score
from sklearn.decomposition import PCA

In [13]:
from configparser import ConfigParser
cfg = ConfigParser()
cfg.read("config.cfg")

['config.cfg']

In [5]:
from tqdm._tqdm_notebook import tqdm_notebook as tqdm # we manually import the notebook submodule as the normal one struggles with jupyterlab
tqdm.pandas() # this enables us to use progress_apply instead of apply

In [28]:
sequence_file = os.path.join(cfg.get("directory", "exchange"), "[wikidata]004_sequences.p")
labels_file = os.path.join(cfg.get("directory", "exchange"), "[wikidata]004_labels.p")
results_prefix = "[WIKIDATA]"

In [7]:
df = pd.read_pickle(sequence_file)
labels = pd.read_pickle(labels_file)['label'].values
df.head()

,length,sequence,user_id
0,1523,"[CLAIM_CREATE, CLAIM_CREATE, BREAK, DESCRIPTIO...",1
1,8,"[SITELINK_ADD, ENTITY_CREATE, BREAK, MERGE, BR...",1000036
2,7,"[SITELINK_ADD, BREAK, SITELINK_ADD, BREAK, SIT...",1000078
3,8,"[SITELINK_REMOVE, SITELINK_ADD, BREAK, SITELIN...",100008
4,35,"[SITELINK_ADD, SITELINK_ADD, BREAK, SITELINK_A...",100012


In [8]:
def calc_dist(sequence):
    dist, pivot = lib.calc_distribution(labels, sequence)
    return np.array(dist)

with Pool(cfg.getint("core", "num_cores")) as processor_pool:
    df['stat_dist'] = pd.Series(processor_pool.imap(calc_dist, tqdm(df['sequence'])))

In [9]:
df.head()

,length,sequence,user_id,stat_dist
0,1523,"[CLAIM_CREATE, CLAIM_CREATE, BREAK, DESCRIPTIO...",1,"[0.022328090570691267, 0.01999355101471073, 0...."
1,8,"[SITELINK_ADD, ENTITY_CREATE, BREAK, MERGE, BR...",1000036,"[0.020189392589955717, 0.020189392589955717, 0..."
2,7,"[SITELINK_ADD, BREAK, SITELINK_ADD, BREAK, SIT...",1000078,"[0.01947397679104996, 0.019473976791049966, 0...."
3,8,"[SITELINK_REMOVE, SITELINK_ADD, BREAK, SITELIN...",100008,"[0.020283553057881, 0.020283553057881005, 0.02..."
4,35,"[SITELINK_ADD, SITELINK_ADD, BREAK, SITELINK_A...",100012,"[0.019736842105263157, 0.019736842105263153, 0..."


In [48]:
kernel_range = np.arange(cfg.getint("kmeans", "clusters_min"), cfg.getint("kmeans", "clusters_max") + 1)
kernel_range

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10])

In [51]:
vectors = np.array([np.array(x) for x in df['stat_dist'].values])
pca = PCA(n_components=3)
plot_vectors = pca.fit_transform(vectors)

In [ ]:
centroids = {}
for num_centroids in tqdm(kernel_range):    
    print("Num Centroids: {n}".format(n=num_centroids))
    kmeans = KMeans(n_clusters=num_centroids)
    kmeans.fit_predict(vectors)
    
    lbl = kmeans.labels_
    centroids[num_kernels] = kmeans.cluster_centers_
    
    silhouette_avg = silhouette_score(vectors, lbl)
    print("SILHOUETTE", silhouette_avg)
    calinski_score = calinski_harabaz_score(vectors, lbl)
    print("CALINSKI", calinski_score)
    sample_silhouette_values = silhouette_samples(vectors, lbl)
    
    store_path = os.path.join(cfg.get("directory", "results"), results_prefix + "kmeans[{n}]".format(n=num_centroids))
    cluster_lbl = "kmeans_{n}".format(n=num_centroids)
    df[cluster_lbl] = lbl
    plotting.k_means(plot_vectors, num_centroids, lbl, sample_silhouette_values, silhouette_avg, store_path)

Num Centroids: 2
SILHOUETTE 0.24465049950771547
CALINSKI 26715.340842608748
data/results/[WIKIDATA]kmeans[2]_silhouette.png
data/results/[WIKIDATA]kmeans[2]_silhouette.pdf
data/results/[WIKIDATA]kmeans[2]_clusters.png
data/results/[WIKIDATA]kmeans[2]_clusters.pdf
data/results/[WIKIDATA]kmeans[2]_population.png
data/results/[WIKIDATA]kmeans[2]_population.pdf
Num Centroids: 3
SILHOUETTE 0.31213095622951675
CALINSKI 26486.153542312382
data/results/[WIKIDATA]kmeans[3]_silhouette.png
data/results/[WIKIDATA]kmeans[3]_silhouette.pdf
data/results/[WIKIDATA]kmeans[3]_clusters.png
data/results/[WIKIDATA]kmeans[3]_clusters.pdf
data/results/[WIKIDATA]kmeans[3]_population.png
data/results/[WIKIDATA]kmeans[3]_population.pdf
Num Centroids: 4
SILHOUETTE 0.3227249771967627
CALINSKI 26742.259175355997
data/results/[WIKIDATA]kmeans[4]_silhouette.png
data/results/[WIKIDATA]kmeans[4]_silhouette.pdf
data/results/[WIKIDATA]kmeans[4]_clusters.png
data/results/[WIKIDATA]kmeans[4]_clusters.pdf
data/results/[WIK

In [ ]:
for num_centroids in kernel_range:
    member_count = defaultdict(int)
    cluster = defaultdict(list)
    cluster_lbl = "kmeans_{n}".format(n=num_centroids)
    
    for i, user in tqdm(df.iterrows(), total=len(df)):
        cluster_id = user[cluster_lbl]
        member_count[cluster_id] += 1
        action_df = pd.DataFrame({"from": user['sequence']})
        action_df["to"] = action_df["from"].shift(periods=-1)
        cluster[cluster_id].append(action_df)
        
    for cluster_index in cluster:
        cluster_df = pd.concat(cluster[cluster_index])

        transition_count_pivot = lib.stretch_pivot(pd.crosstab(cluster_df['from'], cluster_df['to']), states)
        unnormalized = transition_count_pivot.transpose().sum().transpose()
        pivot = lib.stretch_pivot(pd.crosstab(cluster_df['from'], cluster_df['to'], normalize="index"), states)
        
        
        store_path = os.path.join(cfg.get("directory", "results"), results_prefix + "clusters[{n}_{i}]".format(n=num_centroids, i=cluster_index))
        plotting.transition_matrix(unnormalized, pivot, transition_count_pivot, cluster_index, store_path)
        
        """
        transition_count_pivot_reduced = lib.stretch_pivot(transition_count_pivot, filtered_states)
        unnormalized_reduced = transition_count_pivot_reduced.transpose().sum().transpose()
        pivot_reduced = lib.stretch_pivot(pd.crosstab(cluster_df['from'], cluster_df['to'], normalize="index"), filtered_states)
        
        store_path = os.path.join(cfg.get("directory", "results"), results_prefix + "clusters[{n}_{i}]reduced".format(n=num_centroids, i=cluster_index))
        plotting.transition_matrix(unnormalized, pivot, transition_count_pivot, cluster_index, store_path)
        """
        